# 네이버 영화평 감성분석 - TfidfVectorizer, Naive Bayes

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('data/naver_movie_train.tsv', sep='\t')
test_df = pd.read_csv('data/naver_movie_test.tsv', sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


>Tokenizer 함수 정의

In [15]:
from konlpy.tag import Okt
okt = Okt()
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 자 와 ㅋㅋ ㅠㅠ ㅎㅎ'
stopwords = stop_text.split()

In [16]:
def okt_tokenizer(text):
    morphs = okt.morphs(text, stem=True)
    tokens = [word for word in morphs if word not in stopwords]
    return tokens

> TfidfVectorizer로 변환

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(
    tokenizer=okt_tokenizer, ngram_range=(1,2), max_df=0.9
)

In [18]:
%time tvect.fit(train_df.document)

Wall time: 3min 25s


TfidfVectorizer(max_df=0.9, ngram_range=(1, 2),
                tokenizer=<function okt_tokenizer at 0x000002D5CBB023A0>)

In [31]:
%time X_train_tv = tvect.transform(train_df.document)

Wall time: 4min 5s


In [30]:
%time X_test_tv = tvect.transform(test_df.document)

Wall time: 1min 21s


In [25]:
y_train = train_df.label.values
y_test = test_df.label.values

>Naive Bayes 분류기

In [26]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [27]:
nb.fit(X_train_tv, y_train)
nb.score(X_test_tv, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [48995, 145791]

> 실제 테스트

In [28]:
reviews = ['아름다운 음악과 아름다운 풍광 그렇지 못한 현실이 찡하네요',
            '메시지와 작위성의 불협화음']

In [29]:
reviews_tv = tvect.transform(reviews)
nb.predict(reviews_tv)

NotFittedError: This MultinomialNB instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.